In [13]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
from dash.dependencies import Input, Output, State
from datetime import datetime, timedelta
from stock import transform_data
import seaborn as sns
import pandas as pd

In [14]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SOLAR], 
                meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}]
                )


In [15]:
current_year = datetime.now().year

In [16]:
month_1 = (datetime.now() - timedelta(days=31)).strftime('%Y-%m-%d')
year_1 = (datetime.now() - timedelta(days=366)).strftime('%Y-%m-%d')
year_5 = (datetime.now() - timedelta(days=(365 * 5  + 1))).strftime('%Y-%m-%d')



In [17]:
sns.set_theme()

In [18]:
df = transform_data("ITUB4.SA", "max")

In [19]:
init_date = df.index.min()
last_date = df.index.max()
line_color = "red" if df.loc[init_date].iloc[0] > df.loc[last_date].iloc[0] else "green"


br_df = pd.read_html("https://www.dadosdemercado.com.br/bolsa/acoes").copy()
tickers = br_df[0]['Ticker']
tickers = [tickers + ".SA" for tickers in tickers]

In [20]:
main_config = {
    'layout': {
        'template': 'simple_white',
        'xaxis': {'showgrid': False},
        'legend': { 'orientation': 'h', 'y': 1, 'yanchor': 'bottom', 'x': 0.5, 'xanchor': 'center'}

    }
}

In [21]:
fig = go.Figure(data=[go.Scatter(x=df.index, y=df.Close)])

In [22]:
app.layout = dbc.Container([
    dbc.Row([
        html.H1("Stock market Analytics",className='text-center text-primary mb-4'),    
        html.Div([
            html.P(
            [
                html.Span(children="Search and see major statistics",
                            style={'fontWeight': 'bold'}),
                html.Span(id='results', className='text-primary', 
                            style={'fontWeight': 'bold'}),
            ],
            style={'display': 'flex', 'justify-content': 'space-between'}
        ),
        
            ]), 
    ]),
    dbc.Row([
        dcc.Dropdown(tickers, "ITUB4.SA", id='dropdown', className="dropdown" ),
        html.Div([
            dbc.ButtonGroup(
                [dbc.Button("1M", id='bb1M',  className="mt-3 ", n_clicks=0),
                 dbc.Button("1Y", id='bb1Y',  className="mt-3 ", n_clicks=0),
                 dbc.Button("5Y", id='bb5Y',  className="mt-3 ", n_clicks=0),
                 dbc.Button("MAX", id='bb_max',  className="mt-3 ", n_clicks=0)]
            ),
        ]),
            
    ]),
    dbc.Row([
        dbc.Col([
            dcc.Graph(
                id='stock-graph',        
                figure = {
                    'data': [
                        go.Scatter(x=df.index, y=df['Close'],
                                    mode='lines', name="ITUB4.SA", line=dict(color=line_color)),
                    ],
                    'layout': go.Layout(
                        **main_config['layout'],
                        title=f'Stock market Data - ITUB4.SA',
                        yaxis={'title': 'Close Values', },
                    ), 
                },style={'margin': '10px 0'}
            ),
        ]),
        dbc.Col([
            dcc.Graph(
                id='return-graph',        
                figure = {
                    'data': [
                        go.Scatter(x=df.index, y=df['Return_Stock'] * 100,
                                    mode='lines', name="Stock Return" ),
                        go.Scatter(x=df.index, y=df['Return_CDI'] * 100,
                                    mode='lines', name="CDI Return")
                    ],
                    'layout': go.Layout(
                        **main_config['layout'],
                        title=f'CDI Return vs ITUB4.SA Return',
                        yaxis={'title': 'Investiments','tickformat': ',.0f', 'ticksuffix':'%'},
                    ), 
                },style={'margin': '10px 0'}
            ),
        ])
    ]),
    
    dbc.Row([
        html.Footer(
            f"Powered by dash - Gustavo Neves {current_year}", className='text-primary'
        )])
])


In [23]:
@app.callback(
    [Output('stock-graph', 'figure'),
    Output('return-graph', 'figure'),
     Output('results', 'children')],
    [Input('bb1M', 'n_clicks'),
     Input('bb1Y', 'n_clicks'),
     Input('bb5Y', 'n_clicks'),
     Input('bb_max', 'n_clicks'),
     Input('dropdown', 'value')
     ],
     
    
)
def update_graph(n_clicks_1M, n_clicks_1Y, n_clicks_5Y, n_clicks_max, selected_df):
    ctx = dash.callback_context
    button_id = ctx.triggered_id
    



    if button_id:
        # Determine which button was clicked
        button_value = button_id.split('.')[0]
        selected_period = "MAX"

        if button_value == 'bb1M':
            selected_period = '1M'
        elif button_value == 'bb1Y':
            selected_period = '1Y'
        elif button_value == 'bb5Y':
            selected_period = '5Y'
        elif button_value == 'bb_max':
            selected_period = 'MAX'
        
        
        if selected_df:
            df = transform_data(selected_df)
            
            if selected_period == '1M':
                df = transform_data(selected_df, None, month_1)
            elif selected_period == '1Y':
                df = transform_data(selected_df, None, year_1)
            elif selected_period == '5Y':
                df = transform_data(selected_df, None, year_5)
            elif button_value == 'bb_max':
                df = df.loc[df.index.min():]

            init_value = df['Close'].iloc[0]
            last_value = df['Close'].iloc[-1]

            line_color = "red" if init_value > last_value else "green"

            figure1 = {
                'data': [
                    go.Scatter(x=df.index, y=df['Close'],
                                mode='lines', name="ITUB4.SA", line=dict(color=line_color)),
                    ],
                'layout': go.Layout(
                    **main_config['layout'],
                    title=f'Stock market Data - {selected_df}',
                    yaxis={'title': 'Close Values'},
                )
            }
            figure2 = {
                'data': [
                    go.Scatter(x=df.index, y=df['Return_Stock'] * 100,
                            mode='lines', name="Stock Return",),
                    go.Scatter(x=df.index, y=df['Return_CDI'] * 100,
                            mode='lines', name="CDI Return", )
                ],
                'layout': go.Layout(
                    **main_config['layout'],
                    title=f'CDI Return vs {selected_df} Return',
                    yaxis={'title': 'Investiments', 'tickformat': ',.0f', 'ticksuffix':'%'},
                ),
            }

            return figure1, figure2, f"Selected period: {selected_period}"
            

    # Return an empty figure if no button is clicked or no df is selected
    if selected_df:
        df = transform_data(selected_df)        
        init_value = df['Close'].iloc[0]
        last_value = df['Close'].iloc[-1]

        line_color = "red" if init_value > last_value else "green"
        figure1 = {
                    'data': [
                        go.Scatter(x=df.index, y=df['Close'],
                                    mode='lines', name="ITUB4.SA", line=dict(color=line_color) )
                    ],
                    'layout': go.Layout(
                        **main_config['layout'],
                        title=f'Stock market Data - ITUB4.SA',
                        yaxis={'title': 'Close Values'},
                    )
        }

        figure2 = {
            'data': [
                go.Scatter(x=df.index, y=df['Return_Stock'] * 100,
                        mode='lines', name="Stock Return",),
                go.Scatter(x=df.index, y=df['Return_CDI'] * 100,
                        mode='lines', name="CDI Return", )
            ],
            'layout': go.Layout(
                **main_config['layout'],
                title=f'CDI Return vs {selected_df} Return',
                yaxis={'title': 'Investiments', 'tickformat': ',.0f', 'ticksuffix':'%'},
            ),
        }

        return figure1, figure2,  None



In [24]:
if __name__ == "__main__":
    app.run_server(port=4050)